# Quantum Key Distribution + "Quantum Computer Virus"

We learn about Quantum Key Distribution <a href="https://qiskit.org/textbook/ch-algorithms/quantum-key-distribution.html">here</a> and in brief, we have Alice and Bob trying to communicate a secret key over a channel/third party, Eve, and Quantum Key Distribution can show that the key are different if Eve is trying to eavesdrop (my bad english, "eavesdrop" doesn't exist but you get the meaning) the conversation. 

However suppose that Eve doesn't exist, and Alice and Bob communicates via Wabinab's channel. And this guy Wabinab are so bored of studying quantum computing (pleading for forgiveness here for whoever reading this) that he decides to create some random "Quantum Computer Virus" and inject this "phase virus" into the communication channel. More technically explained, he create some random numbers and use these random numbers to create random phase gates and when the package reaches his channel, he tries to "interfere" with the original message being communicated in between Alice and Bob. 

Let's see what would happen. 

In [1]:
import qiskit.tools.jupyter
%qiskit_version_table